In [160]:
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import RMSprop
from keras import optimizers
import sys
from keras.callbacks import LambdaCallback
import random
import matplotlib.pyplot as plt
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

In [161]:
tweet_data=pd.read_csv('../Load_Tweets/data/tweet_data.csv')

In [162]:
tweet_text = tweet_data['TEXT']

In [163]:
tweet_text_all = tweet_data['TEXT'].str.cat(sep=' ')

In [164]:
tweet_text_all[0:1000]

'PAY TO PLAY POLITICS. #CrookedHillary [URL] Very little pick-up by the dishonest media of incredible information provided by WikiLeaks. So dishonest! Rigged system! Crooked Hillary Clinton likes to talk about the things she will do but she has been there for 30 years - why didn\'t she do them? Thank you Florida- a MOVEMENT that has never been seen before and will never be seen again. Lets get out &amp;… [URL] Join me Thursday in Florida &amp; Ohio!West Palm Beach, FL at noon:[URL]Cincinnati, OH this 7:30pm:[URL] Wow, @CNN Town Hall questions were given to Crooked Hillary Clinton in advance of big debates against Bernie Sanders. Hillary &amp; CNN FRAUD! Thank you Texas! If you haven\'t registered to VOTE- today is your last day. Go to: [URL] &amp; get ou… [URL] VOTER REGISTRATION DEADLINES TODAY. You can register now at: [URL] and get out to… [URL] DON\'T LET HER FOOL US AGAIN. [URL] Crooked\'s State Dept gave special attention to "Friends of Bill" after the Haiti Earthquake. Unbelieva

In [165]:
print ('Length of text: {} characters'.format(len(tweet_text_all)))

Length of text: 1513769 characters


In [166]:
# Take a look at the first 250 characters in text
print(tweet_text_all[:250])

PAY TO PLAY POLITICS. #CrookedHillary [URL] Very little pick-up by the dishonest media of incredible information provided by WikiLeaks. So dishonest! Rigged system! Crooked Hillary Clinton likes to talk about the things she will do but she has been t


In [167]:
# list_of_words=pd.Series(tweet_text_all.split())
# print ('Length of text: {} words'.format(len(list_of_words)))
# word_vocab=set(list_of_words)
# print ('{} unique words'.format(len(word_vocab)))

In [168]:
import re
tweet_text_all=re.sub('(\w)\.(\w)', '\\1. \\2', tweet_text_all)
tweet_text_all=re.sub('(\w)(\.{2,4})(\w)', ' \\1 \\2 \\3', tweet_text_all)
tweet_text_all=re.sub('(\w)(\.{2,4}) ', '\\1 \\2 ',tweet_text_all) #fdj... fdj ...
tweet_text_all=re.sub(' (\.{2,6})(\w)', ' \\1 \\2',tweet_text_all) #...fdj ... fdj
tweet_text_all=re.sub('[)(]', '', tweet_text_all)


In [169]:
# list_of_words=pd.Series(tweet_text_all.split())
# print ('Length of text: {} words'.format(len(list_of_words)))
# word_vocab=set(list_of_words)
# print ('{} unique words'.format(len(word_vocab)))

In [170]:
tweet_text_all=re.sub('([^.])\.([^.])', '\\1 EOS\\2', tweet_text_all) 
# tweet_text_all = tweet_text_all.replace(".", ' <EOS>')
tweet_text_all[0:1000]

'PAY TO PLAY POLITICS EOS #CrookedHillary [URL] Very little pick-up by the dishonest media of incredible information provided by WikiLeaks EOS So dishonest! Rigged system! Crooked Hillary Clinton likes to talk about the things she will do but she has been there for 30 years - why didn\'t she do them? Thank you Florida- a MOVEMENT that has never been seen before and will never be seen again EOS Lets get out &amp;… [URL] Join me Thursday in Florida &amp; Ohio!West Palm Beach, FL at noon:[URL]Cincinnati, OH this 7:30pm:[URL] Wow, @CNN Town Hall questions were given to Crooked Hillary Clinton in advance of big debates against Bernie Sanders EOS Hillary &amp; CNN FRAUD! Thank you Texas! If you haven\'t registered to VOTE- today is your last day EOS Go to: [URL] &amp; get ou… [URL] VOTER REGISTRATION DEADLINES TODAY EOS You can register now at: [URL] and get out to… [URL] DON\'T LET HER FOOL US AGAIN EOS [URL] Crooked\'s State Dept gave special attention to "Friends of Bill" after the Haiti 

In [171]:
list_of_words=pd.Series(tweet_text_all.split())
print ('Length of text: {} words'.format(len(list_of_words)))

Length of text: 266327 words


In [172]:
#number of unique words
word_vocab=sorted(set(list_of_words))
print ('{} unique words'.format(len(word_vocab)))

27784 unique words


In [173]:
# The unique characters in the file
vocab = sorted(set(tweet_text_all))
print ('{} unique characters'.format(len(vocab)))

367 unique characters


In [174]:
#replace apostrophes in dictionary keys 
for i in range (len(list_of_words)):
    list_of_words[i]=list_of_words[i].replace("‘", '').replace("’", '').replace("'", '')

In [175]:
#see how many words after apostrophe deletion
word_vocab=sorted(set(list_of_words))
print ('{} unique words'.format(len(word_vocab)))

27390 unique words


In [176]:
#create forward and reverse word index
word2idx = dict((c, i) for i, c in enumerate(word_vocab, 1))
# idx2word = dict((i, c) for i, c in enumerate(word_vocab,1 ))
idx2word= np.asarray(word_vocab)

text_as_int = np.array([word2idx[w] for w in list_of_words])

In [177]:
print('{')
for word,_ in zip(word2idx, range(100)):
    print('  {:4s}: {:3d},'.format(repr(word), word2idx[word]))
print('  ...\n}')

{
  '!' :   1,
  '!!':   2,
  '!!!':   3,
  '!!!!!"':   4,
  '!!!"':   5,
  '!!"':   6,
  '!"':   7,
  '"#DonaldTrump':   8,
  '"....the':   9,
  '"20,000📈21,000📈22,000📈':  10,
  '"46%':  11,
  '"77%':  12,
  '"@007cigarjoe:':  13,
  '"@11phenomenon:':  14,
  '"@1lion:':  15,
  '"@1sonny12:':  16,
  '"@55Lidsville:':  17,
  '"@60Minutes:':  18,
  '"@93101Dianne':  19,
  '"@ABC:DonaldTrump:':  20,
  '"@ABCPolitics:':  21,
  '"@AK_TWEET:':  22,
  '"@ARSenMissyIrvin:':  23,
  '"@AZTRUMPTRAIN:':  24,
  '"@AlexNightrasor:':  25,
  '"@AllegrettiVicki:':  26,
  '"@AmFree:':  27,
  '"@AmericanAsPie:':  28,
  '"@AngPiazza:':  29,
  '"@AngieSteinberg:':  30,
  '"@AnnCoulter:':  31,
  '"@AnneBellar:':  32,
  '"@AnnesLimo:':  33,
  '"@AnnetteJeanne:':  34,
  '"@AnyoneTennis:':  35,
  '"@AprilLaJune:':  36,
  '"@AshleyEdam:':  37,
  '"@Ausbiz:':  38,
  '"@AynsFriend:':  39,
  '"@BarronG510:':  40,
  '"@Belizediver88:':  41,
  '"@BertShad:':  42,
  '"@BirgitOlsen1:':  43,
  '"@Blackan:':  44,
  '"@B

In [178]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- words mapped to int ---- > {}'.format(repr(' '.join(list_of_words[:13])), text_as_int[:13]))

'PAY TO PLAY POLITICS EOS #CrookedHillary [URL] Very little pick-up by the dishonest' ---- words mapped to int ---- > [10831 12959 10858 10868  6856   936 14295 13692 20618 22267 15722 25244
 17358]


In [179]:
idx2word

array(['!', '!!', '!!!', ..., '🚨Happening', '🦃', '\U0010fc00'],
      dtype='<U123')

In [180]:
# The maximum length sentence we want for a single input in characters
seq_length = 20
examples_per_epoch = len(list_of_words)//seq_length

# Create training examples / targets
word_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in word_dataset.take(5):
  print(idx2word[i.numpy()])

PAY-FOR-PLAY
TODAY
PLEDGE?
POLL
EOS!


In [185]:
sequences = word_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(' '.join(idx2word[item.numpy()])))  


'PAY-FOR-PLAY TODAY PLEDGE? POLL EOS! #CrookedHillary! [URL]#AZPrimary Very, little, picked by, the#USWomensOpen, dishonest! media! of! incredible! information! provides by, WikiLeaks: EOS!'
'So, dishonest, Rigged! system, Crooked-USED! Hillary! Clinton! likewise to, talk! about! the#USWomensOpen, things! she, will! do! but, she, has, been! there!'
'for! 30% years! -&gt; why! didnt! she, do! theme Thank-You you! Florida-tomorrow a..... MOVEMENT! that! has, never! been! seen! before! and#PoliceWeek'
'will! never! be! seen! again! EOS! Level get! out! &gt;[URL] [URL]#AZPrimary Joined me! Thursday! in! Florida! &amp;, Ohio![URL] Palmer, Bean FL,'
'at! noon:[URL]Warren, OH, this! 7:35 Wow,the @CNN! Towns, Hall- questions! were, gives to, Crooked-USED! Hillary! Clinton! in! advanced of! big! debates!'


In [186]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [187]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(' '.join(idx2word[input_example.numpy()])))
  print ('Target data:', repr(' '.join(idx2word[target_example.numpy()])))

Input data:  'PAY-FOR-PLAY TODAY PLEDGE? POLL EOS! #CrookedHillary! [URL]#AZPrimary Very, little, picked by, the#USWomensOpen, dishonest! media! of! incredible! information! provides by, WikiLeaks:'
Target data: 'TODAY PLEDGE? POLL EOS! #CrookedHillary! [URL]#AZPrimary Very, little, picked by, the#USWomensOpen, dishonest! media! of! incredible! information! provides by, WikiLeaks: EOS!'


In [190]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2word[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2word[target_idx])))

Step    0
  input: 10831 ('PAY-FOR-PLAY')
  expected output: 12959 ('TODAY')
Step    1
  input: 12959 ('TODAY')
  expected output: 10858 ('PLEDGE?')
Step    2
  input: 10858 ('PLEDGE?')
  expected output: 10868 ('POLL')
Step    3
  input: 10868 ('POLL')
  expected output: 6856 ('EOS!')
Step    4
  input: 6856 ('EOS!')
  expected output: 936 ('#CrookedHillary!')


In [191]:
# Batch size 
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 20), (64, 20)), types: (tf.int64, tf.int64)>

In [192]:
embeddings_index = {}
f = open('../word_embeding/glove.twitter.27B.25d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 1193514 word vectors.


In [197]:
# Length of the vocabulary in chars
vocab_size = len(idx2word)+1

# The embedding dimension 
embedding_dim = 25

# Number of RNN units
rnn_units = 1024
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM)) 

for word, i in word2idx.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [199]:
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
  print ("on the GPU BABY!")
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [205]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):

    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                batch_input_shape=[batch_size, None,],
                                trainable=False),
        rnn(rnn_units,
            return_sequences=True, 
            recurrent_initializer='glorot_uniform',
            stateful=True),
        tf.keras.layers.Dense(vocab_size)
      ])
    return model

In [210]:
model = build_model(
  vocab_size = vocab_size, 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

In [211]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 25)            684775    
_________________________________________________________________
gru_1 (GRU)                  (64, None, 1024)          3225600   
_________________________________________________________________
dense_1 (Dense)              (64, None, 27391)         28075775  
Total params: 31,986,150
Trainable params: 31,301,375
Non-trainable params: 684,775
_________________________________________________________________


In [212]:
for input_example_batch, target_example_batch in dataset.take(1): 
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 20, 27391) # (batch_size, sequence_length, vocab_size)


In [219]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [220]:
sampled_indices

array([24569,  1538, 12664, 21825, 19013, 10450, 14280, 22701, 26285,
       26397, 15562, 21108, 11952, 23021,   586, 12241,  8152, 17287,
        3585,  6333])

In [232]:
print("Input: \n", repr(" ".join(idx2word[[input_example_batch[0]]])))
print()
print("Next Char Predictions: \n", repr(" ".join(idx2word[sampled_indices])))

Input: 
 'our..... Nationalist Anthem, 🇺🇸🇯🇵 [URL]#AZPrimary 70% years! ago! today- the#USWomensOpen, Nationalist Security! Council, met, for! the#USWomensOpen, first! time! EOS! Great!'

Next Char Predictions: 
 'spy, #Troublemakers Stapleton order grave Nevada! Zealand, pro-Israel wasted! wheel boy” militarized Rosh railways "Remarkably, Sanford H? disagree @Rockprincess818 DESPICABLE'


In [233]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 20, 27391)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       10.217948


In [234]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

In [235]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [236]:
EPOCHS=3

In [237]:
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/3


AlreadyExistsError: Resource __per_step_0/training/TFOptimizer/gradients/while_1/ReadVariableOp_7/Enter_grad/ArithmeticOptimizer/AddOpsRewrite_Add/tmp_var/N10tensorflow19TemporaryVariableOp6TmpVarE
	 [[{{node training/TFOptimizer/gradients/while_1/ReadVariableOp_7/Enter_grad/ArithmeticOptimizer/AddOpsRewrite_Add/tmp_var}}]] [Op:StatefulPartitionedCall]